In [1]:
# Import BeautifulSoup for parsing and splinter for site navigation
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd 
import requests
import time 

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser("chrome", **executable_path, headless=False)

In [2]:
url = "https://mars.nasa.gov/news/"
browser.visit(url) 

# Scrape page into soup
html = browser.html
soup = bs(html, 'html.parser')

In [3]:
# Save the most recent article, title and date
article = soup.find("div", class_="list_text")
news_p = article.find("div", class_="article_teaser_body").text
news_title = article.find("div", class_="content_title").text
news_date = article.find("div", class_="list_date").text
print(news_date)
print(news_title)
print(news_p)

February 22, 2018
Seven Ways Mars InSight is Different
NASA has a long and successful track record at Mars. Since 1965, it has flown by, orbited, landed and roved across the surface of the Red Planet. What can InSight -- planned for launch in May -- do that hasn’t been done before?


In [4]:
# Find feature image but image is in medium size
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)

html = browser.html
soup = bs(html, 'html.parser')
dataurl = 'https://www.jpl.nasa.gov'+ soup.find('a', class_='button fancybox').get("data-link")
dataurl

'https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA18914'

In [5]:
# Get full sized feature image
browser.visit(dataurl)

html = browser.html
soup = bs(html, 'html.parser')
image = soup.find('img', class_="main_image").get("src")
featured_image = 'https://www.jpl.nasa.gov'+ image
featured_image

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18914_hires.jpg'

In [6]:
# URL of page to be scraped
url3 = 'https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
response = requests.get(url3)

# Create BeautifulSoup object; parse with 'html.parser'
twittersoup = bs(response.text, 'html.parser')

mars_weather = twittersoup.find_all('div', class_="js-tweet-text-container")[0].p.text
mars_weather

'Sol 1974 (Feb 24, 2018), Sunny, high -13C/8F, low -77C/-106F, pressure at 7.29 hPa, daylight 05:37-17:25'

In [7]:
url4 = 'https://space-facts.com/mars/'
marstables = pd.read_html(url4)
marstables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [8]:
marsdf = marstables[0]
marsdf.columns = ['Category', 'Facts']
marsdf.set_index('Category', inplace=True)
html_table = marsdf.to_html()
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Facts</th>    </tr>    <tr>      <th>Category</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [171]:
# Visit the USGS Astogeology site and scrape pictures of the hemispheres
browser = Browser('chrome', headless=False)
url5 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url5)

mars_hemisphere=[]

for i in range (4):
    time.sleep(3)
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = bs(html, 'html.parser')
    link = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ link
    dictionary={"title":img_title,"img_url":img_url}
    mars_hemisphere.append(dictionary)
    browser.back()
    
print(mars_hemisphere)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
